In [8]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
import pandas as pd

label_encoder = LabelEncoder()

In [9]:
dataframe = pd.read_csv('./Dataset/type_7days/Dataset_v1.9.csv')
dataframe.dropna(inplace=True)
dataframe.rename(columns=lambda x : x.lower(), inplace=True)

dataframe['label'] = label_encoder.fit_transform(dataframe['label'])
print(dataframe.shape)
dataframe.head()

(4708, 20)


,id,label,mint_count_per_week,burn_count_per_week,mint_ratio,swap_ratio,burn_ratio,mint_mean_period,swap_mean_period,burn_mean_period,swap_in_per_week,swap_out_per_week,swap_rate,lp_avg,lp_std,lp_creator_holding_ratio,lp_lock_ratio,token_burn_ratio,token_creator_holding_ratio,number_of_token_creation_of_creator
0,0x3cd1c0b98be4451ca51265bbaeb76cf7b31e1c02,0,531.195902,39.839693,0.481928,0.481928,0.036145,0.393718,0.032614,0.507770,304.375252,226.820650,1.338785,0.119760,0.126458,0.000000e+00,0.000000,0.000000e+00,1.302617e-01,1
1,0x0c52de7bb69edd144d58d772fab1cd196919f5ef,0,54.542476,2.001559,0.377163,0.608997,0.013841,0.805150,0.640197,0.551141,49.038189,39.030396,1.240506,0.332226,0.431320,0.000000e+00,0.000000,4.000000e-07,1.000000e-08,1
2,0xa00d47b4b304792eb07b09233467b690db847c91,0,230.772631,110.130106,0.274242,0.594884,0.130874,0.452930,0.053773,0.602639,240.784458,259.806931,0.925000,0.377358,4.649864,0.000000e+00,0.064726,0.000000e+00,0.000000e+00,1
3,0x114e40ba90e9d8b002bc6936e3299daa393017bb,0,58.779514,8.540613,0.103175,0.881834,0.014991,0.759745,0.190998,0.711122,318.012241,184.376765,1.720109,0.467290,4.754048,1.480000e-08,0.693921,3.748588e-03,0.000000e+00,1
4,0x40829a59080a12f16bb8fba22354a6a42c810aab,0,94.726400,12.028749,0.155812,0.824402,0.019786,0.508112,0.002882,0.630434,313.248679,187.949207,1.662234,0.476190,5.466877,0.000000e+00,0.790202,0.000000e+00,1.000000e-08,1


In [10]:
x = dataframe.drop(columns=['id', 'label'], axis=1)
y = dataframe['label']
scaler = StandardScaler().fit(x)
x = pd.DataFrame(scaler.transform(x))

x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [11]:
xgboost_model = XGBClassifier(objective="binary:logistic", n_estimators=20, random_state=42, eval_metric=["auc", "error", "error@0.6"])
xgboost_model.fit(x_train, y_train, eval_set=[(x_test, y_test)])

[0]	validation_0-auc:0.94791	validation_0-error:0.11890	validation_0-error@0.6:0.09448


[1]	validation_0-auc:0.95390	validation_0-error:0.08599	validation_0-error@0.6:0.08705
[2]	validation_0-auc:0.95794	validation_0-error:0.08599	validation_0-error@0.6:0.09130
[3]	validation_0-auc:0.96139	validation_0-error:0.08280	validation_0-error@0.6:0.08599
[4]	validation_0-auc:0.96339	validation_0-error:0.08280	validation_0-error@0.6:0.08493
[5]	validation_0-auc:0.96323	validation_0-error:0.07643	validation_0-error@0.6:0.08174
[6]	validation_0-auc:0.96607	validation_0-error:0.07537	validation_0-error@0.6:0.08386
[7]	validation_0-auc:0.96828	validation_0-error:0.07537	validation_0-error@0.6:0.08280
[8]	validation_0-auc:0.96850	validation_0-error:0.07749	validation_0-error@0.6:0.08386
[9]	validation_0-auc:0.96981	validation_0-error:0.07643	validation_0-error@0.6:0.08280
[10]	validation_0-auc:0.96874	validation_0-error:0.07325	validation_0-error@0.6:0.07749
[11]	validation_0-auc:0.96957	validation_0-error:0.07219	validation_0-error@0.6:0.07537
[12]	validation_0-auc:0.96912	validation_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False,
              eval_metric=['auc', 'error', 'error@0.6'], feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [12]:
y_pred = xgboost_model.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.9309978768577495